# Sliding Blocks

Consider the sliding blocks puzzle where we are given a $3 \times 3$ grid of blocks with each block containing a unique integer between 0 and 8. An example configuration is given below.

1 2 3

7 8 5

0 6 4

At each step, the block containing 0 can swap places with an adjacent block. 


(a)  Use A$^*$ search to start from any given initial configuration and reach the  goal configuration 

 1 2 3

 4 5 6

 7 8 0

with the sum of Manhattan distances of the blocks from their goal positions as the heuristic. The Manhattan distance of a pair of blocks occupying the integer $n$ at the locations $(i,p)$ and $(j,q)$ (where $i,j,p,q \in \{0,\ldots, 2\}$ and $n \in \{0,\ldots,8 \}$) is given by $ |i-j| + |p-q|$. Print the total number of steps taken to reach the goal and the blocks configuration at each step. 

In [48]:
from queue import PriorityQueue

class sliding_blocks:
    '''This class is sliding blocks puzzle'''
    def __init__(self, m, goal):
        self.m = m
        goal_map = dict()
        for i in range(self.m):
            for j in range(self.m):
                goal_map[goal[i][j]] = (self.m - 1 - i, j)
        self.goal_map = goal_map
    
    def MOD(self, value):
        if value >= 0:
            return value
        return -1 * value
    
    # Heuristic 1
    def Manhattan_distance(self, present_state):
        h_n = 0
        for i in range(self.m):
            for j in range(self.m):
                p_row = self.m - 1 - i  # present state row
                p_col = j               # present state column
                
                key = present_state[i][j]
                g_row, g_col = self.goal_map[key] # goal row, goal column
                h_n += MOD(p_row - g_row) + MOD(p_col - g_col)
        return h_n
    
    # Heuristic 2
    def misplaced_blocks(self, present_state):
        h_n = 0
        for i in range(self.m):
            for j in range(self.m):
                p_row = self.m - 1 - i  # present state row
                p_col = j               # present state column
                
                key = present_state[i][j]
                g_row, g_col = self.goal_map[key]
                if p_row != g_row or p_col != g_col:
                    h_n += 1
        return h_n
    
    # Heuristic 3
    def zero_Manhattan_distance(self, present_state):
        h_n = 0
        for i in range(self.m):
            for j in range(self.m):
                if present_state[i][j] == 0:
                    p_row = self.m - 1 - i  # present state row
                    p_col = j               # present state column
                    
                    key = present_state[i][j]
                    g_row, g_col = self.goal_map[key] # goal row, goal column
                    h_n += MOD(p_row - g_row) + MOD(p_col - g_col)
                    return h_n
        return h_n
    
    # Goal test 
    def goal_test(self, present_state):
        for i in range(self.m):
            for j in range(self.m):
                key = present_state[i][j]
                p_row = self.m -1 - i     # present state row
                p_col = j                 # present state col
                
                g_row, g_col = self.goal_map[key]
                
                if p_row != g_row or p_col != g_col:
                    return False
        return True
    
    def find_neighbours(self, present_state):
        neighbours = []
        zero_row = -1
        zero_col = -1
        
        flag = False
        for i in range(self.m):
            for j in range(self.m):
                row = self.m - 1 - i
                col = j
                
                if present_state[i][j] == 0:
                    zero_row = row
                    zero_col = col
                    flag = True
            if flag is True:
                break
        
        # North Direction
        if zero_row != self.m - 1:
            next_state = [list(row) for row in present_state]
            next_state[self.m -1 - zero_row][zero_col], next_state[self.m - 1 - (zero_row + 1)][zero_col] = next_state[self.m - 1 - (zero_row + 1)][zero_col], next_state[self.m -1 - zero_row][zero_col] 
            next_state = tuple(tuple(row) for row in next_state)
            neighbours.append(next_state)
        
        # South Direction
        if zero_row != 0:
            next_state = [list(row) for row in present_state]
            next_state[self.m -1 - zero_row][zero_col], next_state[self.m - 1 - (zero_row - 1)][zero_col] = next_state[self.m - 1 - (zero_row - 1)][zero_col], next_state[self.m -1 - zero_row][zero_col] 
            next_state = tuple(tuple(row) for row in next_state)
            neighbours.append(next_state)
        
        # West Direction
        if zero_col != 0:
            next_state = [list(row) for row in present_state]
            next_state[self.m -1 - zero_row][zero_col], next_state[self.m - 1 - zero_row][zero_col - 1] = next_state[self.m - 1 - zero_row][zero_col - 1], next_state[self.m -1 - zero_row][zero_col] 
            next_state = tuple(tuple(row) for row in next_state)
            neighbours.append(next_state)
        
        # East Direction
        if zero_col != self.m - 1:
            next_state = [list(row) for row in present_state]
            next_state[self.m -1 - zero_row][zero_col], next_state[self.m - 1 - zero_row][zero_col + 1] = next_state[self.m - 1 - zero_row][zero_col + 1], next_state[self.m -1 - zero_row][zero_col] 
            next_state = tuple(tuple(row) for row in next_state)
            neighbours.append(next_state)
        
        return neighbours
    
    # Astar algo
    def a_star(self, initial_state, ques):
        q = PriorityQueue()
        parent = dict()
        
        # explored = 0
        h_n = self.heuristic(ques, initial_state)
        g_n = 0
        f_n = g_n + h_n
        
        parent[initial_state] = [g_n, None]
        q.put((f_n, initial_state))
        action_cost = 1
        
        isPath = False
        while q.empty() is not True:
            f_n, present_state = q.get()
            parent_g_n = parent[present_state][0]
            
            if self.goal_test(present_state) is True:
                isPath = True
                break
            neighbours = self.find_neighbours(present_state)
            g_n = action_cost + parent_g_n
            for next_state in neighbours:
                g_n = action_cost + parent_g_n
                if parent.get(next_state, False) is False or g_n < parent[next_state][0] :
                    parent[next_state] = [g_n, present_state]
                    h_n = self.heuristic(ques, present_state)
                    f_n = g_n + h_n
                    q.put((f_n, next_state))
        
        path = ''
        if isPath is True:
            node = goal
            temp = ''
            for row in node:
                temp = temp + f'{row}\n'    
            path = temp + '\n'
            while node != source:
                node = parent[node][1]
                temp = ''
                i = 0
                for row in node:
                    if i == 1:
                        temp = temp + f'{row} -->\n'
                    else:
                        temp = temp + f'{row}\n'
                    i += 1
                temp += '\n'
                path = temp + path
        return path, len(parent), isPath
    
    def heuristic(self, ques, present_state):    # Works similar to API
        if ques == 1:
            return self.Manhattan_distance(present_state)
        elif ques == 2:
            return self.misplaced_blocks(present_state)
        elif ques == 3:
            return self.zero_Manhattan_distance(present_state)

In [59]:
# functions initial_state(m) gives a random initial state  

def initial_state(m):
    values = [i for i in range(m ** 2)]
    size = m ** 2
    matrix = [[None for i in range(m)] for j in range(m)]
        
    for i in range(m):
        for j in range(m):
            matrix[i][j] = values.pop(random.randrange(size))
            size -= 1
    
    temp = ()
    for row in matrix:
        temp += (tuple(row),)
    return temp

m = 3

# Generating random initial state
source = initial_state(m)

# Goal
goal = ((1, 2, 3), (4, 5, 6), (7, 8, 0))
print("SOURCE:")
for row in source:
    print(row)

# Creating the Puzzle of size m of a goal
obj = sliding_blocks(m, goal)

SOURCE:
(6, 0, 5)
(1, 3, 2)
(7, 8, 4)


In [60]:
path, explored, isPath = obj.a_star(source, 1)

if isPath is True:
    print(f"There is a path from {source} to {goal}")
    print("PATH:\n", path, sep = "")
else:
    print(f"There is no path from {source} to {goal}")
print('Number of cells explored =', explored)

There is a path from ((6, 0, 5), (1, 3, 2), (7, 8, 4)) to ((1, 2, 3), (4, 5, 6), (7, 8, 0))
PATH:
(6, 0, 5)
(1, 3, 2) -->
(7, 8, 4)

(0, 6, 5)
(1, 3, 2) -->
(7, 8, 4)

(1, 6, 5)
(0, 3, 2) -->
(7, 8, 4)

(1, 6, 5)
(3, 0, 2) -->
(7, 8, 4)

(1, 0, 5)
(3, 6, 2) -->
(7, 8, 4)

(1, 5, 0)
(3, 6, 2) -->
(7, 8, 4)

(1, 5, 2)
(3, 6, 0) -->
(7, 8, 4)

(1, 5, 2)
(3, 0, 6) -->
(7, 8, 4)

(1, 5, 2)
(0, 3, 6) -->
(7, 8, 4)

(1, 5, 2)
(7, 3, 6) -->
(0, 8, 4)

(1, 5, 2)
(7, 3, 6) -->
(8, 0, 4)

(1, 5, 2)
(7, 3, 6) -->
(8, 4, 0)

(1, 5, 2)
(7, 3, 0) -->
(8, 4, 6)

(1, 5, 2)
(7, 0, 3) -->
(8, 4, 6)

(1, 5, 2)
(7, 4, 3) -->
(8, 0, 6)

(1, 5, 2)
(7, 4, 3) -->
(0, 8, 6)

(1, 5, 2)
(0, 4, 3) -->
(7, 8, 6)

(1, 5, 2)
(4, 0, 3) -->
(7, 8, 6)

(1, 0, 2)
(4, 5, 3) -->
(7, 8, 6)

(1, 2, 0)
(4, 5, 3) -->
(7, 8, 6)

(1, 2, 3)
(4, 5, 0) -->
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


Number of cells explored = 2127


# Observation:

1) Every initial state does not have a path to goal.
2) fact: that board positions are divided into two equivalence classes with respect to reachability: (i) those that lead to the goal position and (ii) those that lead to the goal position if we modify the initial board by swapping any pair of adjacent (non-blank) blocks.

# ------------------------------------------------------------------------------------------------------

(b) Repeat part (a) with the following alternative heurisitics.
1. the number of misplaced blocks 
2. the Manhattan distance of the "0" block alone instead of the sum

Compare the performance of the three heuristics using the size of the explored list as the measure. 

In [61]:
# THe numbers of misplaced blocks
path, explored, isPath = obj.a_star(source, 2)

if isPath is True:
    print(f"There is a path from {source} to {goal}")
    print("PATH:\n", path, sep = "")
else:
    print(f"There is no path from {source} to {goal}")
print('Number of cells explored =', explored)

There is a path from ((6, 0, 5), (1, 3, 2), (7, 8, 4)) to ((1, 2, 3), (4, 5, 6), (7, 8, 0))
PATH:
(6, 0, 5)
(1, 3, 2) -->
(7, 8, 4)

(6, 5, 0)
(1, 3, 2) -->
(7, 8, 4)

(6, 5, 2)
(1, 3, 0) -->
(7, 8, 4)

(6, 5, 2)
(1, 0, 3) -->
(7, 8, 4)

(6, 0, 2)
(1, 5, 3) -->
(7, 8, 4)

(6, 2, 0)
(1, 5, 3) -->
(7, 8, 4)

(6, 2, 3)
(1, 5, 0) -->
(7, 8, 4)

(6, 2, 3)
(1, 5, 4) -->
(7, 8, 0)

(6, 2, 3)
(1, 5, 4) -->
(7, 0, 8)

(6, 2, 3)
(1, 0, 4) -->
(7, 5, 8)

(6, 2, 3)
(1, 4, 0) -->
(7, 5, 8)

(6, 2, 0)
(1, 4, 3) -->
(7, 5, 8)

(6, 0, 2)
(1, 4, 3) -->
(7, 5, 8)

(0, 6, 2)
(1, 4, 3) -->
(7, 5, 8)

(1, 6, 2)
(0, 4, 3) -->
(7, 5, 8)

(1, 6, 2)
(4, 0, 3) -->
(7, 5, 8)

(1, 0, 2)
(4, 6, 3) -->
(7, 5, 8)

(1, 2, 0)
(4, 6, 3) -->
(7, 5, 8)

(1, 2, 3)
(4, 6, 0) -->
(7, 5, 8)

(1, 2, 3)
(4, 0, 6) -->
(7, 5, 8)

(1, 2, 3)
(4, 5, 6) -->
(7, 0, 8)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


Number of cells explored = 10469


In [63]:
path, explored, isPath = obj.a_star(source, 3)

if isPath is True:
    print(f"There is a path from {source} to {goal}")
    print("PATH:\n", path, sep = "")
else:
    print(f"There is no path from {source} to {goal}")
print('Number of cells explored =', explored)

There is a path from ((6, 0, 5), (1, 3, 2), (7, 8, 4)) to ((1, 2, 3), (4, 5, 6), (7, 8, 0))
PATH:
(6, 0, 5)
(1, 3, 2) -->
(7, 8, 4)

(0, 6, 5)
(1, 3, 2) -->
(7, 8, 4)

(1, 6, 5)
(0, 3, 2) -->
(7, 8, 4)

(1, 6, 5)
(7, 3, 2) -->
(0, 8, 4)

(1, 6, 5)
(7, 3, 2) -->
(8, 0, 4)

(1, 6, 5)
(7, 0, 2) -->
(8, 3, 4)

(1, 0, 5)
(7, 6, 2) -->
(8, 3, 4)

(1, 5, 0)
(7, 6, 2) -->
(8, 3, 4)

(1, 5, 2)
(7, 6, 0) -->
(8, 3, 4)

(1, 5, 2)
(7, 0, 6) -->
(8, 3, 4)

(1, 5, 2)
(7, 3, 6) -->
(8, 0, 4)

(1, 5, 2)
(7, 3, 6) -->
(8, 4, 0)

(1, 5, 2)
(7, 3, 0) -->
(8, 4, 6)

(1, 5, 2)
(7, 0, 3) -->
(8, 4, 6)

(1, 5, 2)
(7, 4, 3) -->
(8, 0, 6)

(1, 5, 2)
(7, 4, 3) -->
(0, 8, 6)

(1, 5, 2)
(0, 4, 3) -->
(7, 8, 6)

(1, 5, 2)
(4, 0, 3) -->
(7, 8, 6)

(1, 0, 2)
(4, 5, 3) -->
(7, 8, 6)

(1, 2, 0)
(4, 5, 3) -->
(7, 8, 6)

(1, 2, 3)
(4, 5, 0) -->
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


Number of cells explored = 55678


# Observation:
    As Number of cells explored is of order : Manhattan_heuristic < Misplaced_blocks < zero_Manhattan_distance
    so performance : Manhattan_heuristic > Misplaced_blocks > zero_Manhattan_distance